In [44]:
from conllu import parse, parse_incr
import pandas as pd
import os

In [45]:
DATASET_PATH = 'D:/Files/Datasets/UD_Russian-SynTagRus-master'

In [46]:
data_file = open(os.path.join(DATASET_PATH, 'ru_syntagrus-ud-train-a.conllu'), 'r', encoding='utf-8')

# for i in range(20):
#     print(parse_incr(data_file).__next__())

print(parse_incr(data_file).__next__())
print(type(parse_incr(data_file).__next__()))

TokenList<Анкета, ., metadata={sent_id: "2003Anketa.xml_1", text: "Анкета."}>
<class 'conllu.models.TokenList'>


In [47]:
parsed_list = list(parse_incr(data_file))

In [63]:
a = parsed_list[0][1]
a

{'id': 2,
 'form': 'приемной',
 'lemma': 'приемная',
 'upos': 'NOUN',
 'xpos': None,
 'feats': {'Animacy': 'Inan',
  'Case': 'Loc',
  'Gender': 'Fem',
  'Number': 'Sing'},
 'head': 6,
 'deprel': 'obl',
 'deps': [('obl:в:loc', 6)],
 'misc': None}

In [57]:
for token in parsed_list[0]:
    print(token)

В
приемной
его
с
утра
ожидали
посетители
,
-
кое-кто
с
важными
делами
,
а
кое-кто
и
с
такими
,
которые
легко
можно
было
решить
в
нижестоящих
инстанциях
,
не
затрудняя
Семена
Еремеевича
.


In [49]:
parsed_list[0].metadata.items()

dict_items([('sent_id', '2003Anketa.xml_3'), ('text', 'В приемной его с утра ожидали посетители, - кое-кто с важными делами, а кое-кто и с такими, которые легко можно было решить в нижестоящих инстанциях, не затрудняя Семена Еремеевича.')])

In [50]:
for key, value in parsed_list[0].metadata.items():
    print(key, '  ', value)

sent_id    2003Anketa.xml_3
text    В приемной его с утра ожидали посетители, - кое-кто с важными делами, а кое-кто и с такими, которые легко можно было решить в нижестоящих инстанциях, не затрудняя Семена Еремеевича.


In [51]:
metadata = ', '.join(f"{key}: \"{value}\"" for key, value in parsed_list[0].metadata.items())